# Импорт.

In [1]:
from sys import setrecursionlimit
from sys import path
path.append("../../MyMethods")
setrecursionlimit(5000)


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import pickle
import joblib

from Classes.CartTreeClassifier import CartTreeClassifier
from Classes.CartTreeTeacher import CartTreeTeacher

from Functions.metrics import *
from Functions.split_functions import *
from Functions.spreading_functions import *
from Functions.search_split_functions import *

from Functions.metrics import accuracy


# Загрузка Dataset'а.

In [3]:
global ctt

In [4]:
data = pd.read_csv('../Data/data_for_train.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('hazardous', axis=1), data['hazardous'], test_size=0.25, random_state=42)
X_train.shape


In [5]:
y_test

30724     0.0
35732     0.0
118948    1.0
76136     0.0
36621     0.0
         ... 
78301     1.0
156971    1.0
21441     0.0
98558     1.0
77859     0.0
Name: hazardous, Length: 131194, dtype: float64

In [6]:
X_train

,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude
141239,-0.981476,-0.675772,0.590152,-0.151944
63113,-0.986259,-0.574666,0.795103,-0.098039
29924,-0.994915,-0.449857,-0.118752,0.081352
109159,-0.986257,-0.716450,-0.881387,-0.098033
64320,-0.992601,-0.507073,-0.517601,0.013767
...,...,...,...,...
119879,-0.981295,-0.635733,-0.532257,-0.153718
103694,-0.984032,-0.714345,0.317797,-0.125175
131932,-0.992591,-0.636823,-0.081927,0.013857
146867,-0.989026,-0.652895,-0.262361,-0.057075


In [7]:
y_test.shape, y_train.shape, X_train.shape

((131194,), (32798,), (32798, 4))

In [8]:
g, p, n = search_params_that_equiv_to_majority_class(X_train.to_numpy()[0:50], y_train.to_numpy()[0:50])

In [9]:
g, p, n

(0.275, 0.079682937004589, 3)

In [10]:
y_train.to_numpy()[0:50][X_train.to_numpy()[0:50].T[n]>p].shape[0]

17

In [11]:
y_train.to_numpy()[0:50][X_train.to_numpy()[0:50].T[n]<=p].shape[0]

33

# Cart Tree Classifier

In [12]:
global ctt

In [13]:
def split_f(a, b):
    return classic_split(a, b[1], b[2])


In [14]:
def stop(a, b):
    return a**2 and b > 0


In [15]:
ctc = CartTreeClassifier()
ctt = CartTreeTeacher(ctc, 
                      split_f,
                      search_params_that_equiv_to_majority_class, 
                      stop, 1)
ctt.teach(X_train.to_numpy()[:100], y_train.to_numpy()[:100])


In [16]:
ctc

In [17]:
ctt.creation_of_tree


<bound method CartTreeTeacher.creation_of_tree of <Classes.CartTreeTeacher.CartTreeTeacher object at 0x000001EB96FE8D30>>

In [18]:
z = ctc.predict(X_test.to_numpy())

In [19]:
print(
    f'TP : {TruePositive(z, y_test)}',
    f'TN : {TrueNegative(z, y_test)}',
    f'FN : {FalseNegative(z, y_test)}',
    f'FP : {FalsePositive(z, y_test)}',
    f'Accuracy : {accuracy(z, y_test)}',
    f'Specificity : {specificity(z, y_test)}',
    f'Prcision : {precision(z, y_test)}',
    f'Recall : {recall(z, y_test)}',
    f'F1 : {f1(z, y_test)}',
    sep='\n'
)


TP : 64956
TN : 48654
FN : 591
FP : 16993
Accuracy : 0.8659694803115996
Specificity : 0.7411458253994851
Prcision : 0.7926393244578945
Recall : 0.9909835690420614
F1 : 0.8807832076802082


In [20]:
ctc

In [21]:
y_train

141239    1.0
63113     0.0
29924     0.0
109159    1.0
64320     0.0
         ... 
119879    1.0
103694    1.0
131932    1.0
146867    1.0
121958    1.0
Name: hazardous, Length: 32798, dtype: float64

In [22]:
n = 3
for x in X_train.to_numpy():
    k = ctc.tree
    while type(k) is not np.float64:
        print(k.get_all())
        k = k.next(x)
    print(k)


{True: 0.0, False: <Classes.Node.Node object at 0x000001EBFF6435E0>}
{True: 1.0, False: 1.0}
1.0
{True: 0.0, False: <Classes.Node.Node object at 0x000001EBFF6435E0>}
{True: 1.0, False: 1.0}
1.0
{True: 0.0, False: <Classes.Node.Node object at 0x000001EBFF6435E0>}
0.0
{True: 0.0, False: <Classes.Node.Node object at 0x000001EBFF6435E0>}
{True: 1.0, False: 1.0}
1.0
{True: 0.0, False: <Classes.Node.Node object at 0x000001EBFF6435E0>}
{True: 1.0, False: 1.0}
1.0
{True: 0.0, False: <Classes.Node.Node object at 0x000001EBFF6435E0>}
0.0
{True: 0.0, False: <Classes.Node.Node object at 0x000001EBFF6435E0>}
0.0
{True: 0.0, False: <Classes.Node.Node object at 0x000001EBFF6435E0>}
{True: 1.0, False: 1.0}
1.0
{True: 0.0, False: <Classes.Node.Node object at 0x000001EBFF6435E0>}
0.0
{True: 0.0, False: <Classes.Node.Node object at 0x000001EBFF6435E0>}
0.0
{True: 0.0, False: <Classes.Node.Node object at 0x000001EBFF6435E0>}
{True: 1.0, False: 1.0}
1.0
{True: 0.0, False: <Classes.Node.Node object at 0x000

In [32]:
with open('../Models/Tree.pickle', 'wb') as handle:
    b = pickle.dump(ctc.tree.next(X_train.to_numpy()[0]).next(X_train.to_numpy()[0]), handle)


In [30]:
joblib.dump(ctc.tree.next(X_train.to_numpy()[0]), '../Models/Tree.sav')


PicklingError: Can't pickle <function CartTreeTeacher.creation_of_tree.<locals>.f at 0x000001EB96FEF550>: it's not found as Classes.CartTreeTeacher.CartTreeTeacher.creation_of_tree.<locals>.f